In [5]:
import pandas as pd
import numpy as np
import random
import pickle
import sys
from keras.models import load_model
from keras import backend
from keras.models import  Model
from keras import layers

sys.path.append("../../src/features")
import build_features
import RFV_text_vectorizing
sys.path.append("../../src/models/train_model")
import NN_VE_model
sys.path.append("../../src/models/predict_model")
import prediction_util

pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_columns = None

##  Read models and data

In [6]:
# read tokenization values
with open("../../models/cdc_2009_att_text_tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)
with open("../../models/cdc_2009_att_text_max_length.pickle", "rb") as f:
    max_seq_length= pickle.load(f) 
with open("../../models/cdc_2009_att_text_vocabulary.pickle", "rb") as f:
    vocabulary= pickle.load(f) 

In [7]:
# read model
cdc_model = load_model('../../models/cdc_2009_nn_att_text_embedding.H5',custom_objects={'backend': backend})

/home/zenobiosky/anaconda2/lib/python2.7/site-packages/keras/engine/topology.py:1269: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [8]:
fileName = '../../data/processed/FOR_PREDICTIONS_2009.csv'
cdc_file = pd.read_csv(fileName)

In [7]:
len(cdc_file)

19942

In [38]:
#getting one record from the input file, as an example
record_index = random.randint(0,len(cdc_file))
sample_record = cdc_file.iloc[[record_index]]
sample_record.head()

,VYEAR,VMONTH,VDAYR,AGE,ARRTIME,WAITTIME,LOV,RESIDNCE,SEX,ETHUN,RACEUN,ARREMS,TEMPF,PULSE,RESPR,BPSYS,BPDIAS,POPCT,ONO2,GCS,IMMEDR,PAIN,SEEN72,RFV1,RFV2,RFV3,EPISODE,INJURY,CHF,DIABETES,LEFTBMSE,LEFTAMSE,LEFTAMA,DOA,DIEDED,TRANPSYC,TRANOTH,ADMITHOS,OBSHOS,OBSDIS,OTHDISP,ADMIT,HDSTAT,BDATEFL,IMMEDRFL,REGION,MSA
4845,2009,12,4,58,245,425,585,1,1,2,2,1,980,86,20,133,64,97,2,-9,3,0,2,71400,11302,-9,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,-7,-7,0,0,3,1


## Pre-process data

In [39]:
reload(build_features)
reload(RFV_text_vectorizing)
predictors_pre = build_features.get_features(sample_record, with_features_for_Embedding=True, with_target=False)


Creating text for embeddings


In [40]:
predictors_pre

,Temp_Baseline,Pulse_Baseline,Sys_BP_Baseline,Resp_Rate_Baseline,Oxygen_Sat_Baseline,Reason_Chest_Pain,Reason_Abdominal_Pain,Reason_Headache,Reason_Shortness_of_Breath,Reason_Back_Pain,Reason_Cough,Reason_Nausea_Vomiting,Reason_Fever_Chills,Reason_Syncope,Reason_Dizziness,Reason_Psychiatric_Complaint,Reason_Nervous_System,Reason_Cardiovascular_Other,Reason_Ears_Eyes_Complaint,Reason_Respiratory_Other,Reason_Gastrointestinal_Other,Reason_Genitourinary_Other,Reason_Skin_Hair_Nails_Complaint,Reason_Musculoskeletal_Other,Reason_Injury_Poisoning,Reason_Other,Hypothermia,Hyperthermia,Bradycardia,Mild_Tachycardia,Moderate_Tachycardia,Severe_Tachycardia,Hypotension,Hypertension,Bradypnea,Moderate_Tachypnea,Severe_Tachypnea,Mild_Hypoxia,Severe_Hypoxia,Age_18_30,Age_31_40,Age_41_50,Age_51_60,Age_61_70,Age_71_80,Age_81_Above,Male_Flag,Female_Flag,Ambulance_Arrival,Other_Arrival,Unknown_Arrival,ARREMS,AGE,SEX,RFV1,RFV2,RFV3,RFV1_text,RFV2_text,RFV3_text,MSA,DIABETES,CHF,2009_year,2010_year,2011_year,2012_year,2013_year,2014_year,2015_year
0,0.5625,0.086172,0.458621,0.135135,0.135135,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0.392157,0,"[0.7, 0.1, 0.4, 0.0, 0.0]","[0.1, 0.1, 0.3, 0.0, 0.2]","[0, 0, 0, 0, 0]",other reason for visit required by party other...,other,blank entry,1,1,0,1,0,0,0,0,0,0


In [41]:
predictors = RFV_text_vectorizing.vectorize_RFV_text_prediction (predictors_pre, tokenizer,max_seq_length)


## Predict

In [42]:
X_list = NN_VE_model.get_x_list(predictors)
y_pred = cdc_model.predict(X_list).flatten()
y_pred

array([ 0.64905888], dtype=float32)

## Model Interpretability

In [43]:
#cdc_model.summary()

In [46]:
reload(prediction_util)
feature_importance_df, feature_name_list, attention_weights   = \
prediction_util.get_feature_importance (cdc_model,X_list,sample_record,predictors)
text_embeddings_start_post = len(feature_name_list)

In [47]:
TOP = 15
feature_importance_sorted = feature_importance_df.sort_values(by=['attention_weight'], ascending=False).head(TOP)
min_weight = feature_importance_sorted.iloc[TOP-1]["attention_weight"]
word_heat_map = prediction_util.get_rfv_text_heatmap(tokenizer,predictors, \
                                                     attention_weights,min_weight,text_embeddings_start_post )

In [49]:
print 'y_pred: ' , y_pred
feature_importance_sorted 

y_pred:  [ 0.64905888]


,attention_weight,featureName,input_value
1,0.021604,arrival_model,1: by ambulance
4,0.019859,diabetes_indicator,1
59,0.017078,reason2_for_visit,11302:
33,0.017053,Reason_Psychiatric_Complaint,1
44,0.016663,Age_18_30_range,0
58,0.015606,reason1_for_visit,71400: other reason for visit required by part...
41,0.011898,Reason_Musculoskeletal_Other,0
27,0.011212,Reason_Shortness_of_Breath,0
5,0.011180,Age,58
3,0.011068,chf_indicator,0


In [50]:
word_heat_map

,pos_1,pos_2,pos_3,pos_4,pos_5,pos_6,pos_7,pos_8,pos_9,pos_10,pos_11,pos_12,pos_13,pos_14,pos_15,pos_16,pos_17,pos_18,pos_19,pos_20,pos_21,pos_22,pos_23,pos_24,pos_25,pos_26,pos_27,pos_28,pos_29,pos_30,pos_31,pos_32,pos_33,pos_34,pos_35
0,other,reason,for,visit,required,by,party,other,than,the,patient,or,the,healthcare,provider,medical,certificate,physical,certificate,physical,examination,for,adoption,psychiatric,examination,required,by,court,travel,wic,medical,clearance,s,s,other
1,0.00904044,0.00699838,0.006741,0.00799954,0.00688061,0.00674095,0.00699962,0.00904044,0.00699916,0.00708302,0.00647555,0.00672765,0.00708302,0.00700108,0.00699934,0.00651316,0.00690685,0.00678095,0.00690685,0.00678095,0.00796452,0.006741,0.00699889,0.00700297,0.00796452,0.00688061,0.00674095,0.00700081,0.00699874,0.00700208,0.00651316,0.00700201,0.00580302,0.00580302,0.00904044
2,*,,,,,,,*,,,,,,,,,,,,,,,,,,,,,,,,,,,*
